In [1]:
import pandas as pd

In [ ]:
df = pd.concat([pd.read_csv("maestro_vk.csv", index_col=0),
                pd.read_csv("maestro_blog.csv", index_col=0),
                pd.read_csv("maestro_books.csv", index_col=0)])

In [89]:
from razdel import sentenize


def sentify(df):
    df["sentence"] = df.text.apply(lambda y: [x.text for x in sentenize(y)])
    df = df.explode("sentence").reset_index(drop=True).drop("text", axis=1)
    return df

In [120]:
for df_name in [f"maestro_{i}.csv" for i in ["vk", "blog", "books"]]:
    df = pd.read_csv(df_name, index_col=0)
    sentify(df).to_csv(f"sent_{df_name}")

In [121]:
df = pd.concat([pd.read_csv("sent_maestro_vk.csv", index_col=0),
                pd.read_csv("sent_maestro_blog.csv", index_col=0),
                pd.read_csv("sent_maestro_books.csv", index_col=0)])

In [129]:
df = pd.read_csv("sent_combined.csv", index_col=0)
df.head()

,url,title,date,sentence
0,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Начну, простите, с конца."
1,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Да, митинги 10-го и 24-го декабря – это замеча..."
2,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,У России был один приближенный к серьезному ша...
3,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Немного снижая пафос, можно было бы вспомнить ..."
4,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Собственно на этом я бы и завершил статью, одн..."


In [131]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'[^\d\W]+')

In [ ]:
class :
    def __init__(self, v=None):
        self.v = v

Быстрее всего искать что-либо в хешированных структурах, так что в идеале жолжен быть сет, либо хотя бы словрь. Словарь хорош тем, что в него можно сразу затолкать лемму и разбор слова

In [665]:
i = 0
df2 = pd.DataFrame({"token":[], "lemma":[] , "pos":[], "tag":[]})

def preprocess(text):
    global i
    global df2
    
    lowered = tokenizer.tokenize(text.lower())
    forms = predictor.predict(lowered)
    tokens = {}
    lemmas = {}
    for form in forms:
        word = form.word
        lemma = form.normal_form
        
        if word in tokens:
            tokens[word] += [i]
        else:
            tokens.update({word: [i]})
            
        if lemma in lemmas:
            lemmas[lemma] += [i]
        else:
            lemmas.update({lemma: [i]})
            
        df2 = pd.concat([df2, pd.DataFrame({"token": word,
                                            "lemma": lemma,
                                            "pos": form.pos,
                                            "tag": form.tag
                                           },
                                            index=[i])])
        i += 1

    return tokens, lemmas

In [ ]:
# в основном датафрейме храню только список токенов
# по нему будет искаться быстро, если нашёлся, беру номер предложения
# залетаю в токены, беру значение токена, это будет индекс в бд морфологии
# достаю из морфологии всю инфу

In [ ]:
%%time
%%capture

from tqdm import tqdm
tqdm.pandas()

t = pd.DataFrame(df.sentence.apply(preprocess).values)

In [603]:
t["lemmas"] = t[0].apply(lambda x: x["lemmas"])
t["tokens"] = t[0].apply(lambda x: x["tokens"])

In [605]:
t

,0,lemmas,tokens
0,"{'tokens': {'начну': [84], 'простите': [85], '...","{'начать': [84], 'простить': [85], 'с': [86], ...","{'начну': [84], 'простите': [85], 'с': [86], '..."
1,"{'tokens': {'да': [88, 99], 'митинги': [89], '...","{'да': [88, 99], 'митинг': [89], 'го': [90, 92...","{'да': [88, 99], 'митинги': [89], 'го': [90, 9..."


In [ ]:
preprocess(1000) - 1m30s

In [479]:
s = pd.DataFrame(t.tolist(), index= t.index, columns=["tokens", "lemmas", "morph_info"])

In [585]:
t

,0
0,"{'tokens': {'начну': [84], 'простите': [85], '..."
1,"{'tokens': {'да': [88, 99], 'митинги': [89], '..."


In [564]:
word = "начну"

In [662]:
def word_search(word, df=df, df2=df2):
    v = np.vectorize(lambda x: word in x)
    indices = np.where(v(df["tokens"]) == True)[0]
    
    for i in indices:
        print(i)
        print(df.iloc[i]["sentence"])
        morph = df2.iloc[i]
        print(f"""\tnormal form - {morph.lemma}
        pos - {morph.pos}
        tag - {morph.tag}""")
        print("")

In [651]:
print(df.iloc[398]["sentence"])

Конечно, не обошлось и без дипломированных лицедеев – актрисы Натальи Селезневой и актера Валерия Баринова.


In [663]:
word_search(word)

0
Начну, простите, с конца.
	normal form - начать
        pos - VERB
        tag - Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act

308
Начну не с того, что правительство России не выбрало в качестве повода к празднику «народного единства».


IndexError: single positional indexer is out-of-bounds

In [627]:
%%time
v = np.vectorize(lambda x: word in x)

indices = np.where(v(t["tokens"]) == True)

t.iloc[indices]["tokens"].apply(lambda x: df2.iloc[x[word]].values[0])
#chars = s.iloc[indices]["lemmas"].apply(lambda x: x[word])

CPU times: total: 0 ns
Wall time: 998 µs


0    [начну, начать, VERB, Mood=Ind|Number=Sing|Per...
Name: tokens, dtype: object

In [647]:
j = [0, 1]

for i in j:
    print(df.iloc[i]["sentence"])
    morph = df2.iloc[i]
    print(f"""\tnormal form - {morph.lemma}
        pos - {morph.pos}
        tag - {morph.tag}""")

Начну, простите, с конца.
	normal form - начать
        pos - VERB
        tag - Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act
Да, митинги 10-го и 24-го декабря – это замечательная веха отечественной истории, да, прежде всего, это дыхание весны и романтики (я вообще вижу в них, прежде всего, художественный смысл), но (не хочу оказаться правым) ОНИ вам ничего не отдадут.
	normal form - простить
        pos - VERB
        tag - Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act


In [622]:
t.iloc[indices]["tokens"].apply(lambda x: df2["token"].iloc[x[word]])

,84
0,начну


In [464]:
s = pd.concat([df.iloc[:1000].sentence, s], axis=1)

In [465]:
v = np.vectorize(lambda x: (x["pos"], x["tag"]))
v(chars.values)

(array(['VERB', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB',
        'VERB', 'VERB', 'VERB'], dtype='<U4'),
 array(['Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act',
        'Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Plur|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Plur|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act',
        'Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act',
        'Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act',
        'VerbForm=Inf',
        'Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Sing|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Act'],
       dtype='<U66'))

In [ ]:
3 ms - занимает поиск

In [ ]:
# 4 колонки, сначала ищу по датафрейму, это супербыстро с векторайзом, потом в каждой строке ищу индекс один раз, вывожу всю инфу из 4 колонок

In [ ]:
#apply - 188 ms vs vectorize - 15 ms
# подход со словарём уоскрил в 3 раза

In [376]:
indices

(array([0], dtype=int64),)

In [406]:
word = "да"

In [432]:
print(s[["tokens"]].iloc[indices]["tokens"].apply(lambda x: x[word]))
print("!")

1     {'pos': 'CONJ', 'tag': '_'}
12    {'pos': 'CONJ', 'tag': '_'}
22    {'pos': 'CONJ', 'tag': '_'}
30    {'pos': 'CONJ', 'tag': '_'}
47    {'pos': 'CONJ', 'tag': '_'}
Name: tokens, dtype: object
!


In [386]:
# думаю, что так лучше всего, тогда можно по индексу сразу найти предложение, его вывести
# в соотв колонке сразу обратиться по ключу, достать морфоприколы + лемму
# но нужно сделать что-то подобное с другим словарём тоже

# или сделать другую бд, но искать-же всё равно надо по хэшу, а где мне его ещё достать
# обязательно должно что-то на что-то ссылаться
# наверное надо всё-таки хранить лемму в токене тоже и наоборот
# токены надо хранить в сете, иначе я их буду долго искать

# судя по всему список позов тоже нужно хранить, но там нужен только сет,
# чтобы достать форму+поз я посмотрю по словарю


А если всё-таки всё разбить? Искать слово. Допустим, нашёл, вернул индекс, написал предложение + всё, что просят. Вроде так максимально просто. Если поиск в пандасе такой же быстрый как в словаре

In [198]:
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language="ru")

In [204]:
forms = predictor.predict(["мама", "мыла", "раму"])
print(forms[0].pos)
print(forms[0].tag)
print(forms[0].normal_form)
forms[0].word

1/1 [==============================] - 0s 20ms/step
NOUN
Case=Nom|Gender=Fem|Number=Sing
мама


'мама'